In [1]:
!pip install transformers torch
!pip install underthesea


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.3 MB/s eta 0:00:00


In [2]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
from tqdm import tqdm
from underthesea import word_tokenize

In [3]:
# Tải PhoBERT tokenizer và model từ Hugging Face
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert_model = AutoModel.from_pretrained("vinai/phobert-base")


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [4]:
data = pd.read_csv('/kaggle/input/tk2-deepleaning/df_final (1).csv')
# Hàm xử lý văn bản sử dụng Underthesea để tách từ
def preprocess_text(text):
    # Tách từ với underthesea
    text = word_tokenize(text, format="text")
    return text
data['cleaned_old_data'] = data['cleaned_old_data'].astype(str)
data = data[data['cleaned_old_data'].notnull()]
# Áp dụng tiền xử lý lên tập dữ liệu trước khi mã hóa bằng PhoBERT
data['cleaned_old_data'] = data['cleaned_old_data'].apply(preprocess_text)

data

,Unnamed: 0,cleaned_old_data,label
0,0,ngày lần hai nhất ít_định mua bạn nếu ok đều t...,2
1,1,đáng rất giao khi trước kiểm_khách cho gửi tin...,1
2,2,rồi muốn quá thấy về mới không hàng bảo tgdd t...,0
3,3,tiền đồng gần siêu còn chụp tới camera đúng xà...,1
4,4,em bán luôn người khó_thoại điện_xài mà lớn cho,0
...,...,...,...
71269,71269,pin mua bảo_tem là vậy_phẩm sao_hành nhưng phò...,5
71270,71270,lâu ít rồi còn bóc mình dùng nữa thích đã nên ...,5
71271,71271,khách khi hơn tốt nên chuyển nhưng để tâm chốn...,5
71272,71272,yếu thì mua_định như khách_hàng mới về hơn với...,5


In [5]:
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_old_data'], data['label'], test_size=0.2, random_state=42)


In [6]:
# Hàm mã hóa văn bản
def encode_texts(texts, tokenizer, max_length=256):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=max_length)
    return inputs

# Mã hóa dữ liệu
X_train_encoded = encode_texts(X_train.tolist(), tokenizer)
X_test_encoded = encode_texts(X_test.tolist(), tokenizer)

# Chuyển đổi nhãn từ pandas Series sang tensor PyTorch
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)


In [7]:
# Tạo DataLoader cho tập train và test
train_data = TensorDataset(X_train_encoded['input_ids'], X_train_encoded['attention_mask'], y_train)
test_data = TensorDataset(X_test_encoded['input_ids'], X_test_encoded['attention_mask'], y_test)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

In [8]:
class LSTMPhoBERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size=128, num_classes=6):
        super(LSTMPhoBERTClassifier, self).__init__()
        self.bert = bert_model
        self.lstm = nn.LSTM(input_size=768, hidden_size=hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # Bidirectional nên hidden_size * 2

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        lstm_output, (hn, cn) = self.lstm(bert_output)
        # Chỉ lấy hidden state của lớp cuối cùng
        output = self.fc(lstm_output[:, -1, :])
        return output

# Khởi tạo mô hình
model = LSTMPhoBERTClassifier(bert_model=phobert_model, num_classes=6)

# Đưa mô hình sang GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LSTMPhoBERTClassifier(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

In [9]:
# Khởi tạo các thành phần cho quá trình huấn luyện
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# Định nghĩa hàm huấn luyện với hiển thị tiến trình
def train_model(model, train_loader, criterion, optimizer, device, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        print(f"Epoch {epoch+1}/{epochs}")
        
        # Sử dụng tqdm để hiển thị tiến trình theo từng batch
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            input_ids, attention_mask, labels = [x.to(device) for x in batch]

            # Reset gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)

            # Backpropagation
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} completed with average loss: {avg_loss:.4f}")


In [10]:
train_model(model, train_loader, criterion, optimizer, device, epochs=20)

Epoch 1/20


Training Epoch 1: 100%|██████████| 3564/3564 [13:03<00:00,  4.55it/s]


Epoch 1 completed with average loss: 1.5756
Epoch 2/20


Training Epoch 2: 100%|██████████| 3564/3564 [13:12<00:00,  4.50it/s]


Epoch 2 completed with average loss: 1.3965
Epoch 3/20


...

Epoch 18 completed with average loss: 1.0480
Epoch 19/20


Training Epoch 19: 100%|██████████| 3564/3564 [13:10<00:00,  4.51it/s]


Epoch 19 completed with average loss: 1.0375
Epoch 20/20


Training Epoch 20: 100%|██████████| 3564/3564 [13:10<00:00,  4.51it/s]

Epoch 20 completed with average loss: 1.0278


In [11]:
def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            
            outputs = model(input_ids, attention_mask)
            preds = torch.argmax(outputs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return all_labels, all_preds

y_true, y_pred = evaluate_model(model, test_loader, device)

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.49      0.51      2354
           1       0.69      0.57      0.62      2428
           2       0.48      0.43      0.45      2364
           3       0.53      0.55      0.54      2386
           4       0.63      0.61      0.62      2358
           5       0.59      0.79      0.68      2365

    accuracy                           0.57     14255
   macro avg       0.57      0.57      0.57     14255
weighted avg       0.57      0.57      0.57     14255



In [12]:
# Hàm mã hóa câu văn bản đơn lẻ và thực hiện dự đoán
def classify_sentence(model, sentence, tokenizer, device):
    # Mã hóa câu đầu vào bằng PhoBERT
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt", max_length=256)
    
    # Chuyển các tensor đầu vào sang thiết bị (GPU hoặc CPU)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Đặt mô hình ở chế độ đánh giá (evaluation mode)
    model.eval()

    with torch.no_grad():
        # Dự đoán nhãn cho câu đầu vào
        outputs = model(input_ids, attention_mask)
        predicted_label = torch.argmax(outputs, dim=1).item()
    
    return predicted_label


In [13]:
sentence = "giao hàng đầy đủ đóng gói chắc đủ chắn"
predicted_label = classify_sentence(model, sentence, tokenizer, device)
print(f"Đánh giá: {predicted_label}")

Đánh giá: 4
